# Planning Dual-Arm Scenarios

In this notebook we test the motion planning for several dual-arm scenarios.

We create two dual-arm Drake environments: 
* **default**: without obstacles
* **cylinder** with a cylinder obstacle between the robots.

In [ ]:
import numpy as np
from pydrake.planning import RobotDiagramBuilder, SceneGraphCollisionChecker
from cloth_tools.drake.building import add_meshcat_to_builder, finish_build
from cloth_tools.drake.scenes import add_dual_ur5e_and_table_to_builder
from cloth_tools.ompl.dual_arm_planner import DualArmOmplPlanner
from cloth_tools.drake.visualization import publish_dual_arm_joint_path
from airo_models import cylinder_urdf_path
from pydrake.math import RigidTransform

In [ ]:
home_joints_left = np.deg2rad([180, -135, 95, -50, -90, -90])
home_joints_right = np.deg2rad([-180, -45, -95, -130, 90, 90])

In [ ]:
# Creating the default scene
robot_diagram_builder = RobotDiagramBuilder()
meshcat = add_meshcat_to_builder(robot_diagram_builder)
arm_indices, gripper_indices = add_dual_ur5e_and_table_to_builder(robot_diagram_builder)
diagram, context = finish_build(robot_diagram_builder, meshcat)

collision_checker = SceneGraphCollisionChecker(
    model=diagram,
    robot_model_instances=[*arm_indices, *gripper_indices],
    edge_step_size=0.125,  # Arbitrary value: we don't use the CheckEdgeCollisionFree
    env_collision_padding=0.005,
    self_collision_padding=0.005,
)

is_state_valid_default_fn = collision_checker.CheckConfigCollisionFree

In [ ]:
from functools import partial

# Create convenience functions for publishing to default meshcat
publish_path_default = partial(
    publish_dual_arm_joint_path,
    meshcat=meshcat,
    diagram=diagram,
    context=context,
    arm_left_index=arm_indices[0],
    arm_right_index=arm_indices[1],
)

In [ ]:
is_state_valid_default_fn(np.concatenate([home_joints_left, home_joints_right]))

In [ ]:
plant = diagram.plant()
plant_context = plant.GetMyContextFromRoot(context)

arm_left_index, arm_right_index = arm_indices
plant.SetPositions(plant_context, arm_left_index, home_joints_left)
plant.SetPositions(plant_context, arm_right_index, home_joints_right)
diagram.ForcedPublish(context)

In [ ]:
# Creating the scene with a cylinder obstacles
robot_diagram_builder = RobotDiagramBuilder()
meshcat = add_meshcat_to_builder(robot_diagram_builder)
arm_indices, gripper_indices = add_dual_ur5e_and_table_to_builder(robot_diagram_builder)

# Adding the cylinder
plant = robot_diagram_builder.plant()
parser = robot_diagram_builder.parser()

cylinder_urdf_path_ = cylinder_urdf_path(0.55, 0.1)
cylinder_index = parser.AddModelFromFile(
    cylinder_urdf_path_,
)

cylinder_frame = plant.GetFrameByName("base_link", cylinder_index)
cylinder_transform = RigidTransform(p=[0.1, 0, 0.65])
plant.WeldFrames(plant.world_frame(), cylinder_frame, cylinder_transform)

# Finish building and get the collision checker
diagram, context = finish_build(robot_diagram_builder, meshcat)

collision_checker = SceneGraphCollisionChecker(
    model=diagram,
    robot_model_instances=[*arm_indices, *gripper_indices],
    edge_step_size=0.125,  # Arbitrary value: we don't use the CheckEdgeCollisionFree
    env_collision_padding=0.005,
    self_collision_padding=0.005,
)

is_state_valid_cylinder_fn = collision_checker.CheckConfigCollisionFree

In [ ]:
# Create convenience functions for publishing to cylinder meshcat
publish_path_cylinder = partial(
    publish_dual_arm_joint_path,
    meshcat=meshcat,
    diagram=diagram,
    context=context,
    arm_left_index=arm_indices[0],
    arm_right_index=arm_indices[1],
)

In [ ]:
plant = diagram.plant()
plant_context = plant.GetMyContextFromRoot(context)

plant.SetPositions(plant_context, arm_left_index, home_joints_left)
plant.SetPositions(plant_context, arm_right_index, home_joints_right)
diagram.ForcedPublish(context)

In [ ]:
is_state_valid_cylinder_fn(np.concatenate([home_joints_left, home_joints_right]))

## Test Scenarios

A scenario here consists of an environment and robot start and goal configurations.

In [ ]:
def plan_and_publish_path(
    is_state_valid_fn, start_joints_left, start_joints_right, goal_joints_left, goal_joints_right, publish_fn
):
    planner = DualArmOmplPlanner(is_state_valid_fn, max_planning_time=10.0)
    path = planner.plan_to_joint_configuration(
        start_joints_left, start_joints_right, goal_joints_left, goal_joints_right
    )
    publish_fn(path, 5.0)

### Scenario 1: twist_base_90

* Drake scene: default
* Start: both robots at home
* Goal: twist the right arm base joint 90 degrees

**Purpose of the scenario**: very easy to solve, serves as a good sanity check whether everything is configured correctly.

In [ ]:
scenario1_goal_joints_right = home_joints_right.copy()
scenario1_goal_joints_right[0] += np.deg2rad(90)

plan_and_publish_path(
    is_state_valid_default_fn,
    home_joints_left,
    home_joints_right,
    None,
    scenario1_goal_joints_right,
    publish_path_default,
)

### Scenario 2: grasp_highest

* Drake scene: default
* Start: both robots at home
* Goal: right arm move to pose to grasp highest point of cloth

In [ ]:
scenario2_goal_joints_right = home_joints_right.copy()
scenario2_goal_joints_right[1] += np.deg2rad(-80)
scenario2_goal_joints_right[3] += np.deg2rad(80)

plan_and_publish_path(
    is_state_valid_default_fn,
    home_joints_left,
    home_joints_right,
    None,
    scenario2_goal_joints_right,
    publish_path_default,
)

### Scenario 3: grasp_lowest

* Drake scene: cylinder
* Start: left at hanging pose, right at home
* Goal: right arm move to pose to grasp lowest point of cloth

In [ ]:
scenario3_start_joints_left = np.deg2rad([180, -90, 30, -120, -90, -90])
scenario3_goal_joints_right = np.deg2rad([-135, -110, -80, -175, 45, 0])

plan_and_publish_path(
    is_state_valid_cylinder_fn,
    scenario3_start_joints_left,
    home_joints_right,
    None,
    scenario3_goal_joints_right,
    publish_path_cylinder,
)

### Scenario 4: home_single

* Drake scene: default
* Start: both robots entangled in the middle
* Goal: move the right arm home

In [ ]:
scenario4_start_joint_left = np.deg2rad([0, -90, -90, -90, 90, 0])
scenario4_start_joint_right = np.deg2rad([-136, -116, -110, -133, 40, 0])

plan_and_publish_path(
    is_state_valid_default_fn,
    scenario4_start_joint_left,
    scenario4_start_joint_right,
    None,
    home_joints_right,
    publish_path_default,
)

### Scenario 5: home_dual

* Drake scene: default
* Start: both robots entangled in the middle
* Goal: move both arms home

In [ ]:
plan_and_publish_path(
    is_state_valid_default_fn,
    scenario4_start_joint_left,
    scenario4_start_joint_right,
    home_joints_left,
    home_joints_right,
    publish_path_default,
)